### Step 1: Import needed libraries and user-defined functions (UDFs)

The first step is nearly identical to Step 1 in the `read_csv_to_df_operations.ipynb`. There are two additions: 
- import the user-defined function, `create_boolean_cols`
- import `requests`, which helps us programmatically make calls to APIs 

In [2]:
import pandas as pd
import git
import os
import numpy as np
import requests

from utils import create_boolean_cols

In order to import any function located within a different file in your project, you need to insert an `__init__.py` file, which is usually empty, in the working directory. For a more in-depth description of `__init__.py`, this [blog post](https://timothybramlett.com/How_to_create_a_Python_Package_with___init__py.html) provides some useful background.

### Step 2: Fetch the API response from the (Colorado Information Marketplace) CIM
We will utilize the [CIM API](https://dev.socrata.com/foundry/data.colorado.gov/nxw4-ev8w) to bring in data instead of reading in a CSV that is saved locally.

If you look at the *Alternative Energy Laws and Incentives in Colorado 2014* dataset in CIM, you'll notice the below popup when clicking the **API** button in the top right-hand corner

![CIM](images/popup.png)

<hr/>The API endpoint specified in the screenshot is found here:<br/><br/>
https://data.colorado.gov/resource/nxw4-ev8w.json
<br/><br/>
When navigating to this webpage in your browser, you should see the same data as is contained in the `Alternative_Energy_Laws_and_Incentives_in_Colorado_2014.csv` file, just in a different format. This format is called JSON (JavaScript Object Notation), and is extremely common in web applications.

In [3]:
api_url = "https://data.colorado.gov/resource/nxw4-ev8w.json"

first_api_resp = requests.get(api_url)

print(first_api_resp.status_code)

200


When calling `requests.get()`, a [`Response`](https://requests.readthedocs.io/en/master/api/#requests.Response) object, which has a number of _attributes_ and _methods_ associated with it. `status_code` is an attribute since it describes the `Response` object. Status codes `2xx` means that the API request returned successfully. See [this link](https://restfulapi.net/http-status-codes/) for a full list of response codes.

### Step 3: Extract data from the (successful) API response
When we get a 200 status code from a GET request - as we did in the previous cell - we can use a _method_ called `.json()` to extract the data from the response. You can tell `.json()` is a method instead of an attribute like `status_code` because of the parentheses.

In [8]:
json_data = first_api_resp.json()
print(type(json_data))
print(len(json_data))

<class 'list'>
26


From the output of the two print statements, we see the JSON returned from the API request is a list (aka an _array_), and has a length of 26. Each element of the list is a law enacted by CO, same as the CSV from the previous step.

From the docs, we see that we can create a [`pandas.DataFrame()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) from an array using the `data` keyword.

In [9]:
co_energy_laws_df = pd.DataFrame(data=json_data)
co_energy_laws_df.head()

,lawid,title,text,enacteddate,amendeddate,isrecent,sequencenumber,type,technologycategories,incentivecategories,usercategories,references,significantupdatedate,regulationcategories,expireddate
0,6290,Biofuels Research Grants,The Colorado Office of Economic Development ad...,2007-05-03 00:00:00 UTC,2013-05-15 00:00:00 UTC,FALSE,20,State Incentives,ETH|BIOD,GNT,OTHER,http://www.lexisnexis.com/hottopics/michie/,NaN,NaN,NaN
1,11489,Biogas Production Sales Tax Exemption,NaN,2014-05-17 00:00:00 UTC,NaN,FALSE,41,State Incentives,OTHER|BIOD|NG,EXEM,PURCH|AFS|AFP,NaN,2014-07-15 19:05:20 UTC,NaN,NaN
2,11490,Vehicle Fleet Maintenance and Fuel Cost-Saving...,NaN,2013-06-05 00:00:00 UTC,NaN,FALSE,59,Laws and Regulations,ETH|BIOD|ELEC|NG|HEV|LPG,NaN,FLEET,http://www.lexisnexis.com/hottopics/Colorado/,NaN,OTHER|REQ,NaN
3,10014,Alternative Fuel Resale and Generation Regulat...,A corporation or individual that resells alter...,2012-05-03 00:00:00 UTC,NaN,FALSE,58,Laws and Regulations,ELEC|NG|HEV|LPG,NaN,STATION|AFP|AFS|IND|FLEET|PURCH,http://www.lexisnexis.com/hottopics/Colorado/,NaN,OTHER|STD,NaN
4,6578,Plug-In Electric Vehicle (PEV) and Electric Ve...,The Colorado Energy Office (CEO) and Regional ...,2009-06-04 00:00:00 UTC,2014-04-11 00:00:00 UTC,FALSE,15,State Incentives,ELEC,GNT,STATION|IND|FLEET,http://www.leg.state.co.us/clics/clics2012B/cs...,2014-07-15 16:11:19 UTC,NaN,NaN


This is the exact same dataframe we imported from the CSV.

### Step 4: Run operations on the data frame to make it suitable for analysis
We can use the `create_boolean_cols` function we imported earlier to use the raw data to prep our data for analysis.

In [10]:
augmented_co_energy_laws_df = create_boolean_cols(
    co_energy_laws_df, 
    "incentivecategories", #col_name
    "incentive"
)

augmented_co_energy_laws_df = create_boolean_cols(
    augmented_co_energy_laws_df, 
    "regulationcategories", #col_name
    "reg"
)

augmented_co_energy_laws_df = create_boolean_cols(
    augmented_co_energy_laws_df, 
    "usercategories", #col_name
    "user"
)


-------------- start column name: incentivecategories
unique values:
['OTHER', 'GNT', 'TAX', 'EXEM']
building out new boolean columns...
-------------- end column name: incentivecategories

-------------- start column name: regulationcategories
unique values:
['DREST', 'FUEL', 'REGIS', 'STD', 'REQ', 'CCEINIT', 'OTHER']
building out new boolean columns...
-------------- end column name: regulationcategories

-------------- start column name: usercategories
unique values:
['AFP', 'STATION', 'MAN', 'IND', 'PURCH', 'FLEET', 'AFS', 'OTHER']
building out new boolean columns...
-------------- end column name: usercategories


In [12]:
augmented_co_energy_laws_df[["usercategories", "user_FLEET_flg", "user_AFS_flg"]].head()

,usercategories,user_FLEET_flg,user_AFS_flg
0,OTHER,False,False
1,PURCH|AFS|AFP,False,True
2,FLEET,True,False
3,STATION|AFP|AFS|IND|FLEET|PURCH,True,True
4,STATION|IND|FLEET,True,False
5,STATION|FLEET,True,False
6,AFP|PURCH|AFS,False,True
7,IND|FLEET,True,False
8,STATION,False,False
9,IND,False,False


All looks good!

### Comprehension Question
#### How many laws implement fuel taxes? (FUEL in the original `regulationcategories` column)
Use the new columns we created, along with a method discussed earlier, to get the answer